In [3]:
%run ItemModel.ipynb
%run APIService.ipynb

In [4]:
import inspect, re
import pandas as pd
import csv

# Class DataCrawler

In [5]:
class DataCrawler(apiServiceDelegate):
    newest = 0
    keyword = "thoi trang nam"
    categoryids = ""
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
 
    APIService = APIService(newest, keyword, categoryids)
    
    i = 1
    def __init__(self, keyword, newest = 0, categoryids = ''):
        self.newest = newest
        self.keyword = keyword
        self.categoryids = categoryids
        self.APIService = APIService(self.newest, self.keyword, self.categoryids)
        self.APIService.delegate = self
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def crawlData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                check = self.APIService.get()
                if check == False: break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.ctime = item['ctime'] #time since 1970
            itemModel.time_now = int(time.time())
            itemModel.sell_time = itemModel.time_now - itemModel.ctime
            itemModel.historical_sold = item['historical_sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            if len(item['tier_variations']) > 0:
                itemModel.options = 1
                for tier_variation in item['tier_variations']:
                    itemModel.options *= len(tier_variation['options'])
            else:
                itemModel.options = 0
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.five_star = itemModel.rating_count[5]
            itemModel.four_star = itemModel.rating_count[4]
            itemModel.three_star = itemModel.rating_count[3]
            itemModel.two_star = itemModel.rating_count[2]
            itemModel.one_star = itemModel.rating_count[1]
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'category_id\tname\tshop_location\titem_public_time\titem_be_got_time\tsell_time\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\tfive_star\tfour_star\tthree_star\ttwo_star\tone_star\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discount\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'historical_sold\n')

        for item in self.all_items:
                file.write(f'{self.categoryids}\t{item.name}\t{item.shop_location}\t{item.ctime}\t{item.time_now}\t{item.sell_time}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.five_star}\t'
                + f'{item.four_star}\t{item.three_star}\t{item.two_star}\t{item.one_star}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Crawling Data

In [7]:
crawler = DataCrawler(newest = 0, keyword = 'thoi trang nam', categoryids=15139)

In [8]:
crawler.crawlData()


17
get! 1
17
get! 2
17
get! 3
17
get! 4
17
get! 5
17
get! 6
17
get! 7
17
get! 8
17
get! 9
17
get! 10
17
get! 11
17
get! 12
17
get! 13
17
get! 14
17
get! 15
17
get! 16
17
get! 17
17
get! 18
17
get! 19
17
get! 20
Error when request: Expecting value: line 1 column 1 (char 0)


In [82]:
# crawler.parseData()

In [8]:
# Check last request
print(json.dumps(crawler.APIService.json_pydata, indent = 3, sort_keys=True))

{
   "adjust": {
      "count": 248133
   },
   "algorithm": "eyJzZWFyY2giOiI3LmEuMjExQEZQU1VWV1haYWQwTE0xIiwiYWRzIjoib2dZQSJ9",
   "disclaimer_infos": [],
   "error": null,
   "hint_keywords": [],
   "items": [
      {
         "add_on_deal_info": null,
         "ads_keyword": null,
         "adsid": null,
         "badge_icon_type": 0,
         "brand": null,
         "bundle_deal_info": null,
         "campaign_stock": null,
         "campaignid": null,
         "can_use_bundle_deal": false,
         "can_use_wholesale": false,
         "catid": 78,
         "cb_option": 1,
         "cmt_count": 0,
         "coin_earn_label": null,
         "collection_id": null,
         "ctime": 1597198110,
         "currency": "VND",
         "deduction_info": null,
         "discount": null,
         "display_name": null,
         "distance": null,
         "exclusive_price_info": null,
         "flag": 0,
         "flash_sale": null,
         "group_buy_info": null,
         "has_group_buy_stoc

In [22]:
# Check feature value
crawler.all_items[0].ctime

1602518724

In [9]:
# Test ParseData
print(crawler.all_items[0].name)
print(crawler.all_items[50].name)
print(crawler.all_items[1].name)
print(crawler.all_items[51].name)

Áo Polo Embroidered Collar nam FEAER form chuẩn, Chất cotton 100%
Áo thun polo in logo thời trang mùa hè năng động cho nam Bu1berr1
Áo phông nam chấm bi dáng slimfit, chuẩn thiết kế hàn quốc, cực tôn dáng, lịch sự, trẻ trung(ACBI)
Áo Thun nam không cổ họa tiết sọc Striped chất bố thun thoáng mát, co dãn - FEAER DENIM


In [10]:
crawler.saveFile("ShopeeItem-ThoiTrangNam-AoNganTayCoCo.tsv")

In [11]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_AoNganTayCoCo_0-949.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df



FileNotFoundError: [Errno 2] File ShoppeItem_thoiTrangNam_AoNganTayCoCo_0-949.tsv does not exist: 'ShoppeItem_thoiTrangNam_AoNganTayCoCo_0-949.tsv'

In [87]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [88]:
del items_df